# Preprocessing Reviews

In [1]:
import pandas as pd
import numpy as np

In [2]:
airline_data=pd.read_csv('airline.csv')

In [3]:
airline_data.head(1)

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1


In [4]:
reviews=airline_data.content.str.lower()

In [14]:
import re
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import word_tokenize

# Remove Numbers
reviews1=[]
for i in reviews:
    reviews1.append(re.sub(r'\d+', '', i))
    
# Remove Punctuation
reviews2=[]
for i in reviews1:
    table = str.maketrans({key: None for key in string.punctuation})
    reviews2.append(i.translate(table))
    
# Remove Whitespaces
reviews3=[]
for i in reviews2:
    reviews3.append(i.strip())
    
# Removing Stop Words
# stop_words = set(stopwords.words('english'))
tokenize_reviews_no_stop_words=[]
for i in reviews3:
    tokens=tokenize(i)
    result=[j for j in tokens if not j in stop_words]
    temp=' '.join(result)
    tokenize_reviews_no_stop_words.append(temp)

TypeError: sequence item 0: expected str instance, Tok found

In [115]:
def preprocess_review(sentence):
    result=re.sub(r'\d+','',sentence)
    table = str.maketrans({key: None for key in string.punctuation})
    result_new=result.translate(table)
    result_new=result_new.strip()
    stop_words = set(stopwords.words('english'))
    tokens=word_tokenize(result_new)
    final=[j for j in tokens if not j in stop_words]
    temp=' '.join(final)
    return temp


# Polarity


In [7]:
from textblob import TextBlob
polarity=[]
numlist=[]
num=1
for i in tokenize_reviews_no_stop_words:
    temp=TextBlob(i).sentiment.polarity
    polarity.append(temp)
    numlist.append(num)
    num=num+1
    
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.scatter(numlist,polarity,label='Polarity')
plt.xlabel('Reviews')
plt.ylabel('Polarity')
plt.legend()

# Sentiment Values

In [8]:
sentiment=[]
for i in polarity:
    if i==0:
        sentiment.append(0)
    elif i<0:
        sentiment.append(-1)
    elif i>0:
        sentiment.append(1)


In [9]:
combined=[]
for i in range(len(sentiment)):
    combined.append([tokenize_reviews_no_stop_words[i],sentiment[i]])

# Making New DataFrame

In [10]:
df=pd.DataFrame(combined,columns=['Reviews','Sentiment'])
df.head()

,Reviews,Sentiment
0,outbound flight fraprn hours min flight though...,1
1,two short hops zrhlju ljuvie fast crj seats co...,1
2,flew zurichljubljana jp newish crj flight almo...,1
3,adria serves min flight ljubljana amsterdam br...,1
4,wawskj economy free snacks drinks star allianc...,1


# Logistic Regression

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
df=shuffle(df)

In [112]:
model=LogisticRegression()
vectorizer=CountVectorizer(ngram_range=(1,2))
x_l=vectorizer.fit_transform(df.Reviews)
model.fit(x_l[:29488],df.Sentiment[:29488].values)
model.score(x_l[29488:],df.Sentiment[29488:].values)

c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9113201209271078

In [123]:
import pickle

In [125]:
pickle.dump(model, open('logisticModel0.1.pkl', 'wb'))
pickle.dump(vectorizer,open('logisticVectorizer0.1.pkl','wb'))

# Linear SVM

In [127]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=42, tol=1e-5)
clf.fit(x_l[:29488],df.Sentiment[:29488].values)

c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=1e-05,
     verbose=0)

In [128]:
clf.score(x_l[29488:],df.Sentiment[29488:])        

0.9114040980853207

In [139]:
from sklearn.calibration import CalibratedClassifierCV
t=preprocess_review('I fly at least once a month ZRH-LJU-ZRH. My experience is that the staff on the ground need a better education regarding how to care of passengers. If it comes to a delay and pax have to go to the info desk to change tickets and so on then you really see of how unfriendly and unprepared staff are. The airplanes could be cleaned better and I do also see some room for improvement for the flight staff.')
te=vectorizer.transform([t])
s=CalibratedClassifierCV(clf)
s.fit(x_l[:29488],df.Sentiment[:29488].values)
s.predict(te)

c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

array([1], dtype=int64)

In [142]:
s.score(x_l[29488:],df.Sentiment[29488:])        

0.9089687604971448

In [143]:
pickle.dump(s,open('svmModel0.1.pkl','wb'))

# Neural Networks

In [3]:
from keras.layers import Dense,Conv1D,MaxPool1D,Embedding,Flatten,Dropout,GRU,LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [7]:
reviews_array=np.array(df.Reviews)
sentiment_array=np.array(df.Sentiment)

NameError: name 'df' is not defined

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(sentiment_array)
encoded_Y = encoder.transform(sentiment_array)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

NameError: name 'sentiment_array' is not defined

In [ ]:
token=Tokenizer()
token.fit_on_texts(reviews_array) 
vocab_size=len(token.word_index) +1
print(vocab_size)

In [ ]:
l = 0
for i in reviews_array:
  l += len(i)
  
avg_length = l/len(reviews_array)
review_training = [x[:int(avg_length)] for x in reviews_array]


In [ ]:
encoded = token.texts_to_sequences(reviews_array)
l = []
for i in encoded:
  l.append(len(i))


In [ ]:
padded_docs = pad_sequences(encoded, maxlen=38, padding='post')

In [21]:
padded_docs

array([[    1,   421,  7196, ...,    75,   530,   458],
       [   72,   108,  4368, ...,     0,     0,     0],
       [   25, 17571, 14760, ...,     0,     0,     0],
       ...,
       [  385,  3650,   867, ...,     0,     0,     0],
       [48801, 23410,   241, ...,   618,   564,   426],
       [  488,    88,   515, ...,    52,   142,     1]])

In [ ]:
n_model1 = Sequential()
n_model1.add(Embedding(48802,64,input_length=38))
n_model1.add(LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=True, return_state=False, go_backwards=False, stateful=False, unroll=False))
n_model1.add(Dropout(0.5))
n_model1.add(LSTM(64,return_sequences=False))
n_model1.add(Dropout(0.5))
# n_model1.add(Dense(1,activation="sigmoid"))
# n_model1.summary()
# n_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.add(Dense(8, input_dim=1, activation='relu'))

n_model1.add(Dense(3, activation='softmax'))
n_model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
hist = n_model1.fit(padded_docs,dummy_y,epochs=5,batch_size=100,validation_split=0.2)

NameError: name 'n_model1' is not defined

In [226]:
encoded1 = token.texts_to_sequences(['I am feeling very sad and pathetic.'])
padded_docs1 = pad_sequences(encoded1, maxlen=38, padding='post')
n_model1.predict(padded_docs1)

array([[9.995233e-01, 5.564765e-05, 4.210277e-04]], dtype=float32)

In [41]:
# from sklearn.model_selection import train_test_split
# reviews=df.Reviews.values
# sentiments=df.Sentiment.values

In [42]:
# reviews_train,reviews_test,sentiments_train,sentiments_test=train_test_split(reviews, sentiments, test_size=0.25, random_state=1000)

In [45]:
# v=CountVectorizer()
# v.fit(reviews_train)
# X_train=v.transform(reviews_train)
# X_test=v.transform(reviews_test)

In [47]:
# input_dim = X_train.shape[1]  # Number of features

# modeln = Sequential()  
# #model4.add(Embedding(48802,64,input_length=38))
# modeln.add(Bidirectional( LSTM(64 ,dropout_U = 0.2, dropout_W = 0.2)))
# modeln.add(Dense(3,activation="softmax"))
# modeln.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# modeln.summary()


c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`
  """


ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [196]:
# from keras.models import Sequential
# from keras import layers

# input_dim = X_train.shape[1]  # Number of features

# # model11 = Sequential()
# # model11.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
# # model11.add(layers.Dense(1, activation='sigmoid'))


In [195]:
# model11.compile(loss='binary_crossentropy', 
#               optimizer='adam', 
#               metrics=['accuracy'])
# model11.summary()

In [192]:
# model11.fit(X_train, sentiments_train,epochs=10,verbose=True,validation_data=(X_test, sentiments_test),batch_size=10)

In [194]:
# model11.evaluate(X_test,sentiments_test,verbose=0)

In [193]:
# t='This is Kunal Verma'
# te=v.transform([t])
# model11.predict(te)

# LSTM new

In [66]:
model4 = Sequential()  
model4.add(Embedding(48802,64,input_length=38))
model4.add(Bidirectional( LSTM(64 ,dropout_U = 0.2, dropout_W = 0.2)))
model4.add(Dense(3,activation="softmax"))
model4.summary()
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\users\kunalverma\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 38, 64)            3123328   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 128)               66048     
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 387       
Total params: 3,189,763
Trainable params: 3,189,763
Non-trainable params: 0
_________________________________________________________________


In [67]:
hist = model4.fit(padded_docs,dummy_y,epochs=3,batch_size=100,validation_split=0.2)

Train on 33116 samples, validate on 8280 samples
Epoch 1/3
33116/33116 [==============================] - 59s 2ms/step - loss: 0.4228 - acc: 0.8262 - val_loss: 0.3817 - val_acc: 0.8300
Epoch 2/3
33116/33116 [==============================] - 57s 2ms/step - loss: 0.2405 - acc: 0.9042 - val_loss: 0.3759 - val_acc: 0.8339
Epoch 3/3
33116/33116 [==============================] - 61s 2ms/step - loss: 0.1810 - acc: 0.9299 - val_loss: 0.4352 - val_acc: 0.8362


In [76]:
encoded1 = token.texts_to_sequences([['Smart and efficient service. Small planes but as a tall man (6ft 4) I did not feel cramped either on the CRJ-200 (AMS-LJU) or the CRJ-900 (LJU-AMS). The cabin crew were friendly and the food for such a short flight was good. Also the price was right.']])
padded_docs1 = pad_sequences(encoded1, maxlen=38, padding='post')
model4.predict(padded_docs1)

array([[0.09248289, 0.8340015 , 0.07351567]], dtype=float32)

In [73]:
result=model4.predict_classes(padded_docs1)

2

In [58]:
model4.predict_classes(padded_docs1)

array([2], dtype=int64)

In [52]:
model4.evaluate(padded_docs,dummy_y)

41396/41396 [==============================] - 15s 357us/step


[0.2542341899096891, 0.8955696202531646]

In [68]:
import pickle
pickle.dump(model4,open('lstmModel0.1.pkl','wb'))
pickle.dump(token,open('lstmToken.pkl','wb'))

# Convolutional Neural Network

In [107]:
model_conv = Sequential()
model_conv.add(Embedding(48802, 100, input_length=38))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPool1D(pool_size=4))
model_conv.add(Flatten())
model_conv.add(Dense(3, activation='softmax'))
model_conv.compile(loss='categorical_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 38, 100)           4880200   
_________________________________________________________________
dropout_23 (Dropout)         (None, 38, 100)           0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 34, 64)            32064     
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 8, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 1539      
Total params: 4,913,803
Trainable params: 4,913,803
Non-trainable params: 0
_________________________________________________________________


In [110]:
model_conv.fit(padded_docs,dummy_y, validation_split=0.2, epochs = 10 ,batch_size=100)

Train on 33116 samples, validate on 8280 samples
Epoch 1/10
33116/33116 [==============================] - 44s 1ms/step - loss: 0.3541 - acc: 0.8378 - val_loss: 0.3921 - val_acc: 0.8279
Epoch 2/10
33116/33116 [==============================] - 41s 1ms/step - loss: 0.2209 - acc: 0.9121 - val_loss: 0.3835 - val_acc: 0.8321
Epoch 3/10
33116/33116 [==============================] - 46s 1ms/step - loss: 0.1342 - acc: 0.9528 - val_loss: 0.4455 - val_acc: 0.8231
Epoch 4/10
33116/33116 [==============================] - 47s 1ms/step - loss: 0.0604 - acc: 0.9823 - val_loss: 0.5728 - val_acc: 0.8240
Epoch 5/10
33116/33116 [==============================] - 50s 2ms/step - loss: 0.0220 - acc: 0.9954 - val_loss: 0.7029 - val_acc: 0.8200
Epoch 6/10
33116/33116 [==============================] - 49s 1ms/step - loss: 0.0084 - acc: 0.9991 - val_loss: 0.7901 - val_acc: 0.8163
Epoch 7/10
33116/33116 [==============================] - 47s 1ms/step - loss: 0.0035 - acc: 0.9998 - val_loss: 0.8596 - val_acc:

In [112]:
encoded1 = token.texts_to_sequences(['Smart and efficient service. Small planes but as a tall man (6ft 4) I did not feel cramped either on the CRJ-200 (AMS-LJU) or the CRJ-900 (LJU-AMS). The cabin crew were friendly and the food for such a short flight was good. Also the price was right.'])
padded_docs1 = pad_sequences(encoded1, maxlen=38, padding='post')
model_conv.predict(padded_docs1)

array([[2.1023651e-07, 2.7695142e-09, 9.9999976e-01]], dtype=float32)

In [113]:
pickle.dump(model_conv,open('cnnModel0.1.pkl','wb'))